In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
print(tf.test.gpu_device_name())
# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True


In [2]:
import os
import pandas as pd

import tensorflow as tf
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
import keras.callbacks as kcall
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# from Eve import Eve

%matplotlib inline

___

In [3]:
train_dir = 'data'

In [4]:
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

data 0
data/InSitu 532
data/Invasive 532
data/Normal 532
data/Benign 532


In [5]:
from keras.preprocessing.image import ImageDataGenerator

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.25)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)


#target_size: Tuple of integers (height, width), default: (256, 256). 
#The dimensions to which all images found will be resized.
target_size = (256, 256)
#target_size = (height, width)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = target_size,       
        class_mode = 'categorical',
        batch_size=32,
        subset="training",
        shuffle = True)

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = target_size,        
        class_mode = 'categorical',
        batch_size=32,
        subset = "validation",
        shuffle = True)

Found 1596 images belonging to 4 classes.
Found 532 images belonging to 4 classes.


In [6]:
## Intilizing variables
output_classes = 4
batch_size = 32 
epochs = 30

# sgd_opt = SGD(lr=1E-2, decay=1E-4, momentum=0.9, nesterov=True)
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-5)
# eve_opt = Eve(lr=1E-4, decay=1E-4, beta_1=0.9, beta_2=0.999, beta_3=0.999, small_k=0.1, big_K=10, epsilon=1e-08)

xception_weights = 'pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [8]:
from keras import applications
from keras import layers
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model


def xception_model():
    input_tensor = Input(shape=(256, 256, 3))
    base_model = applications.Xception(weights = xception_weights, include_top=False, input_tensor=input_tensor)

    y = base_model.output
    y = GlobalAveragePooling2D(name='avg_pool')(y)
    y = Dropout(0.2)(y)

    predictions = Dense(4, activation='softmax', name='predictions')(y)

    model = Model(input=input_tensor, output=predictions)
    print('Model created.')

    train_top_only = False

    if train_top_only:
        for layer in base_model.layers:
            layer.trainable = False
    else:
        for layer in model.layers[:85]:
            layer.trainable = True
        for layer in model.layers[85:]:
            layer.trainable = True

    for i, layer in enumerate(model.layers):
        print(i, layer.name)

    model.summary()

    return model

In [9]:
model = xception_model()
model.compile(loss = 'categorical_crossentropy', optimizer = adam_opt, metrics = ['accuracy'])

Model created.
0 input_1
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv2_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d_1
13 block2_pool
14 batch_normalization_1
15 add_1
16 block3_sepconv1_act
17 block3_sepconv1
18 block3_sepconv1_bn
19 block3_sepconv2_act
20 block3_sepconv2
21 block3_sepconv2_bn
22 conv2d_2
23 block3_pool
24 batch_normalization_2
25 add_2
26 block4_sepconv1_act
27 block4_sepconv1
28 block4_sepconv1_bn
29 block4_sepconv2_act
30 block4_sepconv2
31 block4_sepconv2_bn
32 conv2d_3
33 block4_pool
34 batch_normalization_3
35 add_3
36 block5_sepconv1_act
37 block5_sepconv1
38 block5_sepconv1_bn
39 block5_sepconv2_act
40 block5_sepconv2
41 block5_sepconv2_bn
42 block5_sepconv3_act
43 block5_sepconv3
44 block5_sepconv3_bn
45 add_4
46 block6_sepconv1_act
47 block6_sepconv1
48 block6_sepconv1_bn
49 block6_sepconv2_act
50 block6_sepconv2
51 block6_sepconv2_bn
5

/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


In [10]:
for i, layer in enumerate(model.layers):
    print('Layer: ',i+1,' Name: ', layer.name)

Layer:  1  Name:  input_1
Layer:  2  Name:  block1_conv1
Layer:  3  Name:  block1_conv1_bn
Layer:  4  Name:  block1_conv1_act
Layer:  5  Name:  block1_conv2
Layer:  6  Name:  block1_conv2_bn
Layer:  7  Name:  block1_conv2_act
Layer:  8  Name:  block2_sepconv1
Layer:  9  Name:  block2_sepconv1_bn
Layer:  10  Name:  block2_sepconv2_act
Layer:  11  Name:  block2_sepconv2
Layer:  12  Name:  block2_sepconv2_bn
Layer:  13  Name:  conv2d_1
Layer:  14  Name:  block2_pool
Layer:  15  Name:  batch_normalization_1
Layer:  16  Name:  add_1
Layer:  17  Name:  block3_sepconv1_act
Layer:  18  Name:  block3_sepconv1
Layer:  19  Name:  block3_sepconv1_bn
Layer:  20  Name:  block3_sepconv2_act
Layer:  21  Name:  block3_sepconv2
Layer:  22  Name:  block3_sepconv2_bn
Layer:  23  Name:  conv2d_2
Layer:  24  Name:  block3_pool
Layer:  25  Name:  batch_normalization_2
Layer:  26  Name:  add_2
Layer:  27  Name:  block4_sepconv1_act
Layer:  28  Name:  block4_sepconv1
Layer:  29  Name:  block4_sepconv1_bn
Layer

In [11]:
num_train_samples= len(train_generator.filenames)
num_train_samples

1596

In [12]:
num_val_samples= len(validation_generator.filenames)
num_val_samples

532

In [ ]:
history = model.fit_generator(train_generator,
#         steps_per_epoch =int(np.ceil(num_train_samples * 1.0 / batch_size)),
        steps_per_epoch = 2000,
        epochs = epochs,
        validation_data = validation_generator,
#         validation_steps=int(np.ceil(num_val_samples * 1.0 / batch_size)))
        validation_steps=500)

Epoch 1/30
  30/2000 [..............................] - ETA: 11:35:45 - loss: 1.3959 - acc: 0.2708

In [ ]:
print(history.history.keys())

In [ ]:
history.history["val_loss"]

In [ ]:
history.history["val_acc"]

In [ ]:
history.history["loss"]

In [ ]:
history.history["acc"]

In [ ]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Test Score: ', score[0])
print ('Test Accuracy: ',score[1])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [ ]:
import numpy as np
plt.figure(1)
plt.plot(history.history['loss'],'r')
plt.plot(history.history['val_loss'],'g')
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])

plt.show()

In [ ]:
filename = validation_generator.filenames
truth = validation_generator.classes
label = validation_generator.class_indices
indexlabel = dict((value, key) for key, value in label.items())

In [ ]:
predicts = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size, verbose=1)
predict_class = np.argmax(predicts, axis=1)
errors = np.where(predict_class != truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(truth,predict_class)

labels = []
for k,v in indexlabel.items():
    labels.append(v)
    
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion Matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion Matrix')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, classes=labels,
                      title='Confusion Matrix')

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score 

#and reports metrics with classification_report method
def predict_and_report(gen, model):
    y_true = []
    y_pred = []
    gen.reset()
    for img, label in gen:
        #get true labels for batch and store them
        y_true.extend([int(z[1]) for z in label])
        #Get predictions as probabilities
        batch_pred = model.predict_on_batch(img)
        #turn probabilities to class labels and store
        batch_pred = np.argmax(batch_pred, axis=1)
        y_pred.extend(batch_pred)
        #break loop
        if gen.batch_index == 0:
            break
            
    print('Accuracy:', accuracy_score(y_true, y_pred))
    print('Area Under the Receiver Operating Characteristic Curve:', roc_auc_score(y_true, y_pred)) #Area under the curve
    print(classification_report(y_true, y_pred))

In [ ]:
predict_and_report(train_generator, model)

In [ ]:
predict_and_report(validation_generator, model)

In [ ]:
best_train_acc = max(history.history['acc'])
best_train_acc

In [ ]:
last_train_acc = history.history['acc'][-1]
last_train_acc

In [ ]:
model.save('models/1.Xception-Adam-Model.h5')
model.save_weights('models/1.Xception-Adam-Weights.h5')